# <a id='toc1_'></a>[Loan Default Prediction (Preprocessing)](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Loan Default Prediction (Preprocessing)](#toc1_)    
  - [Libraries](#toc1_1_)    
  - [Read](#toc1_2_)    
  - [Preprocessing](#toc1_3_)    
    - [Split dataset](#toc1_3_1_)    
    - [Feature selection](#toc1_3_2_)    
      - [Unwanted features](#toc1_3_2_1_)    
      - [Null rates](#toc1_3_2_2_)    
      - [Duplicates features](#toc1_3_2_3_)    
      - [Constant and Quasi-constant features](#toc1_3_2_4_)    
      - [Correlated features](#toc1_3_2_5_)    
    - [Transform](#toc1_3_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Libraries](#toc0_)

In [77]:
# Basic python
import re
import sys

# Data manipulation
import polars as pl
import pandas as pd
import datetime as dt
import numpy as np

# Preprocessing
from feature_engine.selection import DropFeatures, DropConstantFeatures, DropDuplicateFeatures, DropCorrelatedFeatures
from sklearn.impute import KNNImputer, SimpleImputer

# appending a path
sys.path.append('../src/')

# Own modules
import helpers as hp

## <a id='toc1_2_'></a>[Read](#toc0_)

In [78]:
# Define file path
path = "../data/raw/"
raw_file = "definitely_not_from_kaggle_loan_default_dataset.csv"

In [79]:
# Read data
df = pl.read_csv(path + raw_file)

In [80]:
df.head(3)

uuid,credit_application_date,default,person_bith_year_month_1_no_aggregation,activity_date_36_month_1_no_aggregation,location_cluster_03_month_1_most_frequent,contract_type_1_month_1_no_aggregation,activity_pattern_35_month_1_no_aggregation,activity_pattern_34_month_12_no_aggregation,activity_pattern_17_month_12_no_aggregation,activity_pattern_02_month_12_no_aggregation,device_age_2_month_1_social_mean,activity_pattern_09_last_12_months_max,client_type_1_month_9_no_aggregation,activity_pattern_07_last_9_months_max,activity_pattern_07_last_12_months_max,line_balance_type_11_last_12_months_max,line_balance_type_09_last_9_months_mean,activity_pattern_21_weekend_month_12_no_aggregation,activity_pattern_09_last_6_months_max,activity_pattern_09_last_9_months_max,line_balance_type_09_last_12_months_mean,activity_pattern_17_month_9_no_aggregation,line_balance_type_01_last_3_months_min,plan_data_month_12_no_aggregation,activity_pattern_25_last_12_months_min,data_usage_pattern_fortnight_1_last_12_months_max,location_work_null_month_9_no_aggregation,line_balance_type_01_last_6_months_min,activity_pattern_21_weekdaynight_month_12_no_aggregation,activity_pattern_03_weekend_month_6_no_aggregation,activity_pattern_25_last_9_months_min,mobility_pattern_03_last_3_months_mean,data_usage_pattern_days_last_6_months_min,activity_pattern_02_month_1_no_aggregation,activity_pattern_02_last_12_months_mean,billing_pattern_03_last_12_months_mean,…,activity_pattern_14_last_12_months_mean,activity_pattern_12_last_12_months_mean,activity_pattern_05_weekdaylight_last_3_months_mean,activity_pattern_14_last_9_months_mean,device_age_3_month_12_no_aggregation,credit_default_home_last_12_months_min,activity_pattern_14_last_6_months_max,device_technology_2_month_1_no_aggregation,activity_pattern_11_month_12_no_aggregation,device_count_1_month_1_no_aggregation,activity_pattern_27_last_9_months_mean,activity_pattern_21_weekdaynight_last_12_months_max,data_usage_pattern_time_8_month_1_social_min,line_balance_type_04_last_6_months_max,mobility_pattern_05_month_6_no_aggregation,data_usage_pattern_time_5_month_1_social_mean,data_usage_pattern_week_1_last_9_months_mean,location_cluster_02_month_9_no_aggregation,credit_default_work_last_6_months_mean,line_balance_type_07_last_9_months_min,data_usage_pattern_fortnight_1_last_9_months_mean,activity_pattern_11_last_9_months_max,activity_pattern_10_month_6_no_aggregation,activity_pattern_10_month_9_no_aggregation,line_months_active_2_last_12_months_mean,data_usage_pattern_days_last_3_months_mean,activity_pattern_10_weekend_last_3_months_mean,data_usage_pattern_fortnight_4_last_6_months_max,activity_pattern_11_weekdaylight_month_1_no_aggregation,activity_pattern_21_weekdaylight_last_12_months_max,location_cluster_02_month_1_no_aggregation,activity_pattern_22_weekend_last_3_months_min,mobility_pattern_05_last_6_months_min,activity_pattern_08_last_9_months_min,activity_pattern_28_last_3_months_mean,device_age_2_month_6_no_aggregation,activity_pattern_11_weekdaynight_month_9_no_aggregation
str,str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ab5191f0-5bf7-48d7-b52b-046316…","""2022-04-29""",1.0,1999.0,"""2019-03-01""","""REGION_5""","""TYPE_2""",-184.249,22.0,-224.048,-27.9704,-104.1362,-180.4017,12.7514,234.0074,-414.8524,null,null,-198.5122,-14.5693,254.6177,null,130.2402,null,-172.1113,-67.5607,24.5808,28.0,null,-270.9978,-361.9366,116.7504,-55.6501,154.7281,312.7052,-64.8508,151.8844,…,-134.2671,177.1795,244.5875,3.7521,null,-5.4105,-151.4378,94.8894,-303.3621,-15.1243,-99.0,348.433,-91.2841,null,-252.5821,-35.0847,415.4933,1140.2032,93.1231,null,-299.9051,309.5891,-411.5654,-115.208,89.8558,-382.5563,-281.5171,-115.3785,150.4016,-250.0021,-727.2061,-410.8231,-107.3805,85.0,-19.0,-

## <a id='toc1_3_'></a>[Preprocessing](#toc0_)

### <a id='toc1_3_1_'></a>[Split dataset](#toc0_)

In [81]:
# Select all rows from the set containing training samples. We'll train and validate our model with this data
target = "default"
known_set = df.filter(pl.col(target).is_not_null())

# As the dataset will be splitted in function of time i'm forced to change the schema
known_set = known_set.with_columns(pl.col("credit_application_date").cast(pl.Date()))

# As there are records with almost every value being null i will handle separatedly those clients.
# Null set must be revisited. Separate if more than 80% of values per client are null
alpha = 0.8
null_set = known_set.filter(pl.sum_horizontal(pl.all().is_null()) >= alpha* len(df.columns))
known_set = known_set.filter(pl.sum_horizontal(pl.all().is_null()) < alpha* len(df.columns))

# # Save both sets
# null_set.write_csv("../data/raw/null_set.csv")
# known_set.write_csv("../data/raw/not_null.csv")

In [82]:
# 10 months for train and 4 months for validation
train = known_set.filter(pl.col("credit_application_date")<dt.datetime(2022,7,1))
test = known_set.filter(pl.col("credit_application_date")>=dt.datetime(2022,7,1))

In [83]:
known_set.shape[0], train.shape[0], test.shape[0]

(18348, 11923, 6425)

In [84]:
# Verify that default rates is similar between datasets
train["default"].value_counts(normalize=True), test["default"].value_counts(normalize=True)

(shape: (2, 2)
 ┌─────────┬────────────┐
 │ default ┆ proportion │
 │ ---     ┆ ---        │
 │ f64     ┆ f64        │
 ╞═════════╪════════════╡
 │ 1.0     ┆ 0.160278   │
 │ 0.0     ┆ 0.839722   │
 └─────────┴────────────┘,
 shape: (2, 2)
 ┌─────────┬────────────┐
 │ default ┆ proportion │
 │ ---     ┆ ---        │
 │ f64     ┆ f64        │
 ╞═════════╪════════════╡
 │ 0.0     ┆ 0.80965    │
 │ 1.0     ┆ 0.19035    │
 └─────────┴────────────┘)

In [85]:
# Create training and test sets separating target feature. Transform to pandas for sklearn compatibility
X_train = train.drop(pl.col("default")).to_pandas()
y_train = train.select(target).to_pandas()
X_test = test.drop(pl.col("default")).to_pandas()
y_test = test.select(target).to_pandas()

### <a id='toc1_3_2_'></a>[Feature selection](#toc0_)

#### <a id='toc1_3_2_1_'></a>[Unwanted features](#toc0_)

In [86]:
# Drop uuid cause is unique and credit_applicatio_date to remove time dependency to the predictions
DF_unwanted = DropFeatures(features_to_drop=["uuid", "credit_application_date"])
# fit the transformer
DF_unwanted.fit(X_train)

X_train = DF_unwanted.transform(X_train)
X_test = DF_unwanted.transform(X_test)


#### <a id='toc1_3_2_2_'></a>[Null rates](#toc0_)

In [87]:
# Retain just features with less than "x (threshold)" rate of null values.
null_threshold = 0.20
null_cols = [col.name for col in (train.drop(pl.col("default")).null_count()/train.shape[0]).select(pl.all().sum() >= null_threshold) if col.all()]
print("Total features filtered with high null values rate: ", len(null_cols))

Total features filtered with high null values rate:  114


In [88]:
DF_nulls = DropFeatures(features_to_drop=null_cols)
# fit the transformer
DF_nulls.fit(X_train)

X_train = DF_nulls.transform(X_train)
X_test = DF_nulls.transform(X_test)

#### <a id='toc1_3_2_3_'></a>[Duplicates features](#toc0_)

In [89]:
# Drop features containing the same values. Corr = 1 
DDF = DropDuplicateFeatures()
DDF.fit(X_train)

X_train =  DDF.transform(X_train)
X_test =  DDF.transform(X_test)

#### <a id='toc1_3_2_4_'></a>[Constant and Quasi-constant features](#toc0_)

In [90]:
# Drop features with low variance 
DCF = DropConstantFeatures(tol = 0.75, missing_values= "ignore")
DCF.fit(X_train)

X_train = DCF.transform(X_train)
X_test = DCF.transform(X_test)

#### <a id='toc1_3_2_5_'></a>[Correlated features](#toc0_)

In [91]:
# Drop correlated features given a threshold. Let's set 0.8 be the limit for pearson method
DCORR= DropCorrelatedFeatures(variables=None, method='pearson', threshold=0.8)
DCORR.fit(X_train)

X_train = DCORR.transform(X_train)
X_test = DCORR.transform(X_test)

### <a id='toc1_3_3_'></a>[Transform](#toc0_)

#### Dtypes

#### Imputer

Impute regions

In [92]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imp_mean.fit(X_train)

X_train = imp_mean.transform(X_train)
X_test= imp_mean.transform(X_test)

Impute continous features

In [93]:
imputer = KNNImputer(n_neighbors=7)
imputer.fit(X_train)

X_train = imp_mean.transform(X_train)
X_test = imp_mean.transform(X_test)

ValueError: could not convert string to float: '2019-03-01'